<a href="https://colab.research.google.com/github/HyeonminNam/TIL/blob/master/NLP_C/200722_Luhn_Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 토큰화

In [ ]:
import konlpy
import re

In [ ]:
text = '''코로나/19/ 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성, 기술‧응용 공동 연구
지난 8일, 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력

코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 ‘코로나19 감염 대응을 위한 의료 인공지능 컨소시엄’이 9월 1일에 출범한다. 

연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.
 
컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학), 김휘영(방사선의과학연구소), 심규원(신경외과), 염준섭(감염내과), 최병욱(영상의학과, 사진), 허진(영상의학과) 교수가 참여했다.

대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.

이 밖에도 전종홍(한국전자통신연구원), 김경훈(정보통신정책연구원), 예종철(한국과학기술원) 교수도 포함됐다.
 
인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고, 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다.

컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회, 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다.
 
한편 오는 7월 24일(금)에는 온라인 워크숍을 개최해 코로나 19 감염대응 의료 인공지능 관련 동향발표 및 연구개발 사례 소개가 있을 예정이다.'''

In [ ]:
from konlpy.tag import Kkma
kkma= Kkma()
kkma_tokens = kkma.morphs(text)

In [ ]:
kkma_tokens
# 고유명사 처리 결과가 좋지 않기 때문에 다른 형태소 분석기 탐색

In [ ]:
from konlpy.tag import Okt
okt= Okt()
okt_tokens = okt.morphs(text)

In [ ]:
okt_tokens
# 고유명사 처리 결과 나쁘지 않기 때문에 okt 활용하여 토큰화

## 중요단어 결정

In [ ]:
# 중복 제거하여 단어 유형 판단
word_type = set(okt_tokens)
freq_dic = {}

# 각 단어 빈도 딕셔너리에 저장
for x in word_type:
    if re.search('[0-9가-힣a-zA-z]', x):
        freq_dic[x] = okt_tokens.count(x)

In [ ]:
total = sum(freq_dic.values())
keyword_lst = []
for word, freq in freq_dic.items():
    if 0.005 < freq/total < 0.5:    # 0.001로 했을 때 제외되는 단어 거의 없어서 비율 조정
        keyword_lst.append(word)

In [ ]:
print(len(word_type))
print(len(keyword_lst))
# 157개 단어 중에서 41개 단어를 keyword로 추출

## 문장 중요도 계산

In [ ]:
# 개별 문장 리스트
sent_lst = text.split('.')
sent_lst = [x.strip() for x in sent_lst]

In [ ]:
sent_lst

In [ ]:
# 문장 중요도 계산
sent_sig_dic = {}
for x in sent_lst:
    if x:
        sent_tokens = okt.morphs(x)
        s_kw_dic = {}
        for idx, token in enumerate(sent_tokens):
            if token in keyword_lst:
                s_kw_dic[token] = idx
        start = min(s_kw_dic.values())    # 윈도우 시작
        end = max(s_kw_dic.values())  # 윈도우 끝
        word_num = end-start+1
        kw_num = len(s_kw_dic)
        sent_sig = kw_num**2/word_num
        sent_sig_dic[x] = sent_sig

In [ ]:
sent_sig_dic

In [ ]:
sent_sig_lst = list(sent_sig_dic.items())
sent_sig_lst.sort(key=lambda x: x[1], reverse=True)

In [ ]:
sent_sig_lst

In [ ]:
# 중요도 상위 3문장 출력
for idx, x in enumerate(sent_sig_lst[:3]):
    print(idx+1, x[0])

## Luhn_Summarize 클래스

In [ ]:
import re
from konlpy.tag import Kkma
from konlpy.tag import Okt

class LuhnSummarize:
    def __init__(self, text):
        self.text = text
        self.minimum = 0.001
        self.maximum = 0.5
        self.keyword_lst = []

    # 키워드 추출
    def keyword(self, minimum, maximum):
        okt = Okt()
        okt_tokens = okt.morphs(self.text)
        # 중복 제거하여 개별 단어 세트 만들기
        word_type = set(okt_tokens)
        freq_dic = {}
        # 각 단어 빈도 딕셔너리에 저장
        for x in word_type:
            if re.search('[0-9가-힣a-zA-z]', x):
                freq_dic[x] = okt_tokens.count(x)

        # 개별 단어 빈도 / 총 빈도 기준으로 키워드 추출
        total = sum(freq_dic.values())
        keyword_lst = []
        self.minimum = minimum
        self.maximum = maximum
        for word, freq in freq_dic.items():
            if self.minimum < freq/total < self.maximum:    # 0.001로 했을 때 제외되는 단어 거의 없어서 비율 조정
                keyword_lst.append(word)
        self.keyword_lst = keyword_lst
        return keyword_lst


    # 문장 중요도 계산
    def keysentence(self):
        okt = Okt()
        # 개별 문장 리스트
        sent_lst = self.text.split('.')
        sent_lst = [x.strip() for x in sent_lst]
        # 문장 중요도 계산
        sent_sig_dic = {}
        for x in sent_lst:
            # 문장이 비어있으면 제외하고 실행
            if x:
                sent_tokens = okt.morphs(x)
                s_kw_dic = {}
                for idx, token in enumerate(sent_tokens):
                    if token in keyword_lst:
                        s_kw_dic[token] = idx
                start = min(s_kw_dic.values())    # 윈도우 시작
                end = max(s_kw_dic.values())  # 윈도우 끝
                word_num = end-start+1
                kw_num = len(s_kw_dic)
                sent_sig = kw_num**2/word_num
                sent_sig_dic[x] = sent_sig
        for idx, x in enumerate(sent_sig_lst[:3]):
            print(idx+1, x[0])

In [ ]:
text = '''코로나/19/ 대응 의료 인공지능 컨소시엄 출범 산학연병 인공지능 전문가 20여 명으로 구성, 기술‧응용 공동 연구
지난 8일, 창립준비 모임…미영상의학회 COVID-19 컨소시엄과도 협력

코로나19 바이러스의 세계적 감염 유행에 효과적으로 대응하기 위해 관련 전문가들이 모인 ‘코로나19 감염 대응을 위한 의료 인공지능 컨소시엄’이 9월 1일에 출범한다. 

연세대 의대, 한국전자통신원, 네이버 등 산학연병 20여 명이 참여하는 컨소시엄에서는 인공지능 기술과 응용 분야를 공동으로 연구하고 개발, 검증할 예정이다. 또 이 인공지능 기술을 적용하기 위해 필요한 데이터를 수집, 공유, 활용한다. 나아가 다양한 산학연병 전문가들이 국내외적으로 협력하고 공동으로 대응해 나갈 예정이다.
 
컨소시엄에는 15개 기관의 20여 명 전문가들이 참여했다. 연세대 의과대학에서는 박유랑(의생명시스템정보학), 김휘영(방사선의과학연구소), 심규원(신경외과), 염준섭(감염내과), 최병욱(영상의학과, 사진), 허진(영상의학과) 교수가 참여했다.

대학군에서는 권인호(동아대), 김남국(울산대), 김윤현(전남대), 김진영(계명대), 박상준(서울대), 신수용(성균관대), 정명진(성균관대), 진광남(서울대), 홍헬렌(서울여대) 교수가 , 산업군에서는 김기환(루닛), 이동훈(카카오), 정규환(뷰노), 최우식(딥노이드), 최정필(코어라인소프트), 하정우(네이버)가 기업을 대표해 동참했다.

이 밖에도 전종홍(한국전자통신연구원), 김경훈(정보통신정책연구원), 예종철(한국과학기술원) 교수도 포함됐다.
 
인공지능 컨소시엄은 지난 7월 8일에 창립 준비 위원 모임을 했고, 연세대 의대 영상의학과 최병욱 교수(사진)가 준비위원장으로 선출됐다.

컨소시엄은 북미영상의학회에서 시작한 COVID-19 RICORD 컨소시엄, 유럽연합에서 시작한 COVID-19 이미징 AI 이니셔티브 등과 함께 국제 협력 활동을 펼칠 계획이며, 코로나19 대응을 위한 AI 핵심표준 개발도 목표로 하고 있다. 아울러 대한의료인공지능학회, 대한영상의학회 등 국내 관련 학회와도 적극 협력할 계획이다.
 
한편 오는 7월 24일(금)에는 온라인 워크숍을 개최해 코로나 19 감염대응 의료 인공지능 관련 동향발표 및 연구개발 사례 소개가 있을 예정이다.'''

In [ ]:
ls_test = LuhnSummarize()

In [ ]:
ls_test.keyword(0.005, 0.5)

In [ ]:
ls_test.keysentence()